In [1]:
import os, json

In [ ]:
gns = [32, 64, 128]
gls = [4, 6, 8]
mns = [128, 256]
mls = [1, 2 ,3]

param_g = {
        'tag':'cg',
        'encoder_type':'graph',
        'encoder_share':True,
        'graph_net_params':{
            'graph':'cg',
            'hidden_dim':0,
            'output_dim':32,
            'n_layer':0
        }
    }

param_m = {
    'tag':'dnn',
    'encoder_type':'mol',
    'encoder_share':True,
    'mol_net_params':{
        'hidden_dim':0,
        'output_dim':32,
        'n_layer':0
    }
}

param_c = {
        'tag':'cg',
        'encoder_type':'concat',
        'encoder_share':True,
        'graph_net_params':{
            'graph':'cg',
            'hidden_dim':0,
            'output_dim':32,
            'n_layer':0
        },
        'mol_net_params':{
            'hidden_dim':0,
            'output_dim':32,
            'n_layer':0
        }
    }

for gn in gns:
    for gl in gls:
        if gn == 128 and gl == 8: continue
        for mn in mns:
            for ml in mls:
                if mn == 256 and ml == 3: continue
                # graph
                for share in ['dual', 'sing']:
                    p = param_g.copy()
                    p.update({'tag':f'{share}_cg_{gn:02d}_{gl}', 
                              'encoder_share':True if share == 'sing' else False})
                    p['graph_net_params'].update({'hidden_dim':gn, 'n_layer':gl})
                    with open(f'params/scratch/graph/{p["tag"]}.json','w') as f:
                        json.dump(p, f, indent=4)

                    # mol                
                    p = param_m.copy()
                    p.update({'tag':f'{share}_dnn_{mn:02d}_{ml}', 
                              'encoder_share':True if share == 'sing' else False})
                    p['mol_net_params'].update({'hidden_dim':mn, 'n_layer':ml})
                    with open(f'params/scratch/mol/{p["tag"]}.json','w') as f:
                        json.dump(p, f, indent=4)
                        
                    # concat
                    p = param_c.copy()
                    p.update({'tag':f'{share}_cg_{gn:02d}.{mn:03d}_{gl}.{ml}', 
                              'encoder_share':True if share == 'sing' else False})
                    p['graph_net_params'].update({'hidden_dim':gn, 'n_layer':gl})
                    p['mol_net_params'].update({'hidden_dim':mn, 'n_layer':ml})      
                    with open(f'params/scratch/concat/{p["tag"]}.json','w') as f:
                        json.dump(p, f, indent=4)
                    print(p['tag'])

In [ ]:
gns = [32, 64, 128]
gls = [4, 6, 8]
mns = [128, 256]
mls = [1, 2 ,3]

param_g = {
        'tag':'cg',
        'encoder_type':'graph',
        'encoder_share':True,
        'graph_net_params':{
            'graph':'cg',
            'hidden_dim':0,
            'output_dim':32,
            'n_layer':0
        }
    }

param_m = {
    'tag':'dnn',
    'encoder_type':'mol',
    'encoder_share':True,
    'mol_net_params':{
        'hidden_dim':0,
        'output_dim':32,
        'n_layer':0
    }
}

param_c = {
        'tag':'cg',
        'encoder_type':'concat',
        'encoder_share':True,
        'graph_net_params':{
            'graph':'cg',
            'hidden_dim':0,
            'output_dim':32,
            'n_layer':0
        },
        'mol_net_params':{
            'hidden_dim':0,
            'output_dim':32,
            'n_layer':0
        }
    }

for gn in gns:
    for gl in gls:
        if gn == 128 and gl == 8: continue
        for mn in mns:
            for ml in mls:
                if mn == 256 and ml == 3: continue
                # graph
                for share in ['dual', 'sing']:
                    p = param_g.copy()
                    p.update({'tag':f'{share}_cg_{gn:02d}_{gl}', 
                              'encoder_share':True if share == 'sing' else False,
                              'pretrained_path':f'/home/jhyang/WORKSPACES/MODELS/fpoly/r100/encoders_cs/graph/cg_{gn:02d}_{gl}'})
                    p['graph_net_params'].update({'hidden_dim':gn, 'n_layer':gl})
                    with open(f'params/finetune/graph/{p["tag"]}.json','w') as f:
                        json.dump(p, f, indent=4)

                    # mol                
                    p = param_m.copy()
                    p.update({'tag':f'{share}_dnn_{mn:02d}_{ml}', 
                              'encoder_share':True if share == 'sing' else False,
                              'pretrained_path':f'/home/jhyang/WORKSPACES/MODELS/fpoly/r100/encoders_cs/mol/dnn_{mn:02d}_{ml}'})
                    p['mol_net_params'].update({'hidden_dim':mn, 'n_layer':ml})
#                    with open(f'params/finetune/mol/{p["tag"]}.json','w') as f:
#                        json.dump(p, f, indent=4)
                        
                    # concat
                    p = param_c.copy()
                    p.update({'tag':f'{share}_cg_{gn:02d}.{mn:03d}_{gl}.{ml}', 
                              'encoder_share':True if share == 'sing' else False,
                              'pretrained_path':f'/home/jhyang/WORKSPACES/MODELS/fpoly/r100/encoders_cs/concat/cg_{gn:02d}.{mn:03d}_{gl}.{ml}'})
                    p['graph_net_params'].update({'hidden_dim':gn, 'n_layer':gl})
                    p['mol_net_params'].update({'hidden_dim':mn, 'n_layer':ml})      
#                    with open(f'params/finetune/concat/{p["tag"]}.json','w') as f:
#                        json.dump(p, f, indent=4)
#                    print(p['tag'])

In [5]:
for model in ['scratch','finetune']:
    root = f'params/{model}'
    base = json.load(open(os.path.join(root, 'defaults_cs.json')))
#    noises = [0, 0.01, 0.05]
    batchs = [4, 8, 16, 32]
    gamma  = [0.75, 0.95, 0.99]
    steps  = [25, 50]
#    for noise in noises:
    for batch in batchs:
        for gam in gamma:
            for step in steps:
                p = base.copy()
                sfx = f'b{batch:02d}_g{gam*100:.0f}_s{step:02d}'
                p.update({
                    'output_path':f'/home/jhyang/WORKSPACES/MODELS/fpoly/r100/{model}_cs/{sfx}',
                    'batch_size':batch,
                    'scheduler_gamma':gam,
                    'scheduler_step_size':step,
                    'learning_rate':0.002,
                })
                with open(os.path.join(root, f'cs/{sfx}.json'), 'w') as f:
                    json.dump(p, f, indent=4)
                print(f'{sfx}.json')

b04_g75_s25.json
b04_g75_s50.json
b04_g95_s25.json
b04_g95_s50.json
b04_g99_s25.json
b04_g99_s50.json
b08_g75_s25.json
b08_g75_s50.json
b08_g95_s25.json
b08_g95_s50.json
b08_g99_s25.json
b08_g99_s50.json
b16_g75_s25.json
b16_g75_s50.json
b16_g95_s25.json
b16_g95_s50.json
b16_g99_s25.json
b16_g99_s50.json
b32_g75_s25.json
b32_g75_s50.json
b32_g95_s25.json
b32_g95_s50.json
b32_g99_s25.json
b32_g99_s50.json
b04_g75_s25.json
b04_g75_s50.json
b04_g95_s25.json
b04_g95_s50.json
b04_g99_s25.json
b04_g99_s50.json
b08_g75_s25.json
b08_g75_s50.json
b08_g95_s25.json
b08_g95_s50.json
b08_g99_s25.json
b08_g99_s50.json
b16_g75_s25.json
b16_g75_s50.json
b16_g95_s25.json
b16_g95_s50.json
b16_g99_s25.json
b16_g99_s50.json
b32_g75_s25.json
b32_g75_s50.json
b32_g95_s25.json
b32_g95_s50.json
b32_g99_s25.json
b32_g99_s50.json


In [3]:
root = f'../params/hp_opt'
batchs = [4, 8, 16, 32]
gamma  = [0.75, 0.95, 0.99]
steps  = [25, 50]
#    for noise in noises:
for batch in batchs:
    for gam in gamma:
        for step in steps:
            tag = f'b{batch:02d}_g{gam*100:.0f}_s{step:02d}'
            p = {
                'tag': tag,
                'batch_size':batch,
                'scheduler_gamma':gam,
                'scheduler_step_size':step,
            }
            with open(os.path.join(root, f'{tag}.json'), 'w') as f:
                json.dump(p, f, indent=4)

In [11]:
fn = './params/pretrain/defaults_cs.json'
p = json.load(open(fn))

for t in p['target_column']:
    p_ = p.copy()
    p_['output_path'] = p_['output_path'].replace('encoders_cs',f'encoders_st/{t}')
    p_['target_column'] = [t]
    fn_ = fn.replace('cs.json',f'st_{t}.json').replace('pretrain','pretrain/st')
    json.dump(p_, open(fn_,'w'))

In [12]:
p['target_column']

['mu',
 'alpha',
 'homo',
 'lumo',
 'gap',
 'R^2',
 'zpve',
 'Cv',
 'U0_pa',
 'U_pa',
 'H_pa',
 'G_pa']